## Charecter RNN
This is my implementation of the Charecter RNN using LSTM and GRU.

Example adopted from [TensorFlow Examples](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/recurrent_network.ipynb)


Chen Chen

12/15/2016